In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from google.cloud import language_v1
from google.cloud.language_v1 import enums
import pandas as pd

In [40]:
pd.set_option("display.max_colwidth", None)

In [8]:
#####
## Train Model
#####

# Read file
df = pd.read_pickle('../data/label_renamed.pkl')

# Fix Text column (from list to string)
df['TextString'] = [' '.join(map(str, l)) for l in df['Text']]

# from sklearn.feature_extraction.text import TfidfTransformer
X_train, X_test, y_train, y_test = train_test_split(df['TextString'], df['Dominant_Topic'], random_state = 0)

# Train
vectorizer = CountVectorizer()
vtrain = vectorizer.fit_transform(X_train)
lrmodel = LogisticRegression(C=0.3, penalty='l2', multi_class='auto', solver='liblinear', max_iter=1000)
lrmodel.fit(vtrain, y_train)

LogisticRegression(C=0.3, max_iter=1000, solver='liblinear')

In [16]:
#####
## Choose tweets for testing
#####
df = pd.read_csv('sentimentTesting.csv')

In [20]:
df.iloc[30]['text']

'RT @JesseFernandez: Scientists: If you consider all the data, clearly humans are causing climate change.\n\nGOP: But if you don’t, we’re not…'

In [9]:
def sample_analyze_sentiment(text_content):
    """
    Analyzing Sentiment in a String

    Args:
      text_content The text content to analyze
    """

    client = language_v1.LanguageServiceClient()
    type_ = enums.Document.Type.PLAIN_TEXT
    language = "en"
    sentence = {"content": text_content, "type": type_, "language": language}
    encoding_type = enums.EncodingType.UTF8

    response = client.analyze_sentiment(sentence, encoding_type=encoding_type)
    #print(type(response))
    return response

In [30]:
# Define the project
def project(df,i):
    # Find Topic
    v_y = vectorizer.transform([df.iloc[i]['text']])
    topic = lrmodel.predict(v_y)
    
    # Find Sentiment
    response = sample_analyze_sentiment(df.iloc[i]['text'])
    score = round(response.document_sentiment.score,3)
    magnitude = round(response.document_sentiment.magnitude,3)
    
    print("Text from tweet: ", df.iloc[i]['text'])
    print("Topic: ", topic)
    print("Score: ", score)
    print("Magnitude: ", magnitude)
    print("Region: ", df.iloc[i]['user_location'])

In [46]:
project(df,7)

Text from tweet:  Anti-pipeline activist found guilty after being barred from mentioning climate change https://t.co/GdpivVFRBn via @HuffPostGreen
Topic:  ['Action/Efforts/Awareness']
Score:  -0.4
Magnitude:  0.4
Region:  San Diego, California USA
